In [15]:
import pandas as pd

# Load CSV file

url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df = pd.read_csv(url)

In [16]:
df["Single Year of Age"] = pd.to_numeric(df["Single Year of Age"], errors="coerce")
df["VALUE"] = pd.to_numeric(df["VALUE"], errors="coerce")

df = df.dropna(subset=["Single Year of Age", "VALUE"])

In [17]:
df.columns

Index(['STATISTIC', 'Statistic Label', 'TLIST(A1)', 'CensusYear',
       'C02199V02655', 'Sex', 'C02076V03371', 'Single Year of Age',
       'C03789V04537', 'Administrative Counties', 'UNIT', 'VALUE'],
      dtype='object')

In [18]:
# Select only males
males = df[df["Sex"] == "male"]

# Total male population
total_males = males["VALUE"].sum()

# Sum of age and population
male_age_times_pop = (males["Single Year of Age"] * males["VALUE"]).sum()

# Weighted mean age
male_weighted_mean_age = male_age_times_pop / total_males

male_weighted_mean_age

/tmp/ipykernel_10604/2423383269.py:11: RuntimeWarning: invalid value encountered in scalar divide
  male_weighted_mean_age = male_age_times_pop / total_males


np.float64(nan)

In [19]:
# Select only females
females = df[df["Sex"] == "Female"]

# Total female population
total_females = females["VALUE"].sum()

# Sum of age and population
female_age_times_pop = (females["Single Year of Age"] * females["VALUE"]).sum()

# Weighted mean age
female_weighted_mean_age = female_age_times_pop / total_females

female_weighted_mean_age

/tmp/ipykernel_10604/3785488419.py:11: RuntimeWarning: invalid value encountered in scalar divide
  female_weighted_mean_age = female_age_times_pop / total_females


np.float64(nan)

In [23]:
male_by_age = males.groupby("Single Year of Age")["VALUE"].sum()
females_by_age = females.groupby("Single Year of Age")["VALUE"].sum()

difference_by_age = male_by_age - females_by_age
difference_by_age

Series([], Name: VALUE, dtype: int64)